In [1]:
!gdown --id 1uV1Fw9QFUofRREiTqCalmq_pUL6R3TjG

Downloading...
From: https://drive.google.com/uc?id=1uV1Fw9QFUofRREiTqCalmq_pUL6R3TjG
To: /content/Dataset.zip
400MB [00:02, 162MB/s]


In [2]:
!unzip Dataset.zip

Archive:  Dataset.zip
  inflating: SLA_matrix.xlsx         
  inflating: delivery_orders_march.csv  


In [3]:
import pandas as pd
import numpy as np

In [17]:
delivery_df = pd.read_csv('delivery_orders_march.csv')

In [5]:
delivery_df.shape

(3176313, 6)

In [6]:
delivery_df.head(5)

,orderid,pick,1st_deliver_attempt,2nd_deliver_attempt,buyeraddress,selleraddress
0,2215676524,1583138397,1.583385e+09,NaN,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,1583309968,1.583463e+09,1.583799e+09,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,1583306434,1.583460e+09,NaN,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,1583419016,1.583556e+09,NaN,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,1583318305,1.583480e+09,NaN,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


# Cleaning the Data

In [18]:
delivery_df.columns

Index(['orderid', 'pick', '1st_deliver_attempt', '2nd_deliver_attempt',
       'buyeraddress', 'selleraddress'],
      dtype='object')

In [19]:
delivery_df.columns = ['orderid', 'pick', 'deliver_attempt_1', 'deliver_attempt_2', 'buyeraddress', 'selleraddress']

In [20]:
delivery_df[['pick', 'deliver_attempt_1', 'deliver_attempt_2']]

,pick,deliver_attempt_1,deliver_attempt_2
0,1583138397,1.583385e+09,NaN
1,1583309968,1.583463e+09,1.583799e+09
2,1583306434,1.583460e+09,NaN
3,1583419016,1.583556e+09,NaN
4,1583318305,1.583480e+09,NaN
...,...,...,...
3176308,1585821728,1.585978e+09,NaN
3176309,1585895011,1.586179e+09,1.586232e+09
3176310,1585826951,1.585965e+09,NaN
3176311,1585891194,1.586230e+09,NaN


In [22]:
# Assuming that the dates provided by the dataset are in UNIX format

def date_parser(columns, df):
    for s in columns:
        df[s] = pd.to_datetime(df[s], unit='s')
        df[s] = df[s].values.astype('datetime64[D]')
    return df

In [23]:
delivery_df = date_parser(['pick', 'deliver_attempt_1', 'deliver_attempt_2'], delivery_df)
delivery_df.head(5)

,orderid,pick,deliver_attempt_1,deliver_attempt_2,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,NaT,"Baging ldl BUENAVISTA,PATAG.CAGAYAN Buagsong,c...",Pantranco vill. 417 Warehouse# katipunan 532 (...
1,2219624609,2020-03-04,2020-03-06,2020-03-10,coloma's quzom CASANAS Site1 Masiyan 533A Stol...,"BLDG 210A Moras C42B 2B16,168 church) Complex ..."
2,2220979489,2020-03-04,2020-03-06,NaT,"21-O LumangDaan,Capitangan,Abucay,Bataan .Bign...","#66 150-C, DRIVE, Milagros Joe socorro Metro M..."
3,2221066352,2020-03-05,2020-03-07,NaT,"616Espiritu MARTINVILLE,MANUYO #5paraiso kengi...","999maII 201,26 Villaruel Barretto gen.t number..."
4,2222478803,2020-03-04,2020-03-06,NaT,L042 Summerbreezee1 L2(Balanay analyn Lot760 C...,G66MANILA Hiyas Fitness MAYSILO magdiwang Lt.4...


In [35]:
def convert_address(dB: pd.core.frame.DataFrame, address_column: str) -> pd.core.series.Series:

    def returnFinalWord(adr: str) -> str:
        proc_ad = adr.split()
        result_loc = proc_ad[len(proc_ad)-1].lower()
        if result_loc == "manila":
            return 0
        elif result_loc == "luzon":
            return 1
        elif result_loc == "visayas":
            return 2
        elif result_loc == "mindanao":
            return 3

    address = pd.DataFrame()
    address['raw_address'] = dB[address_column]
    address[address_column] = address['raw_address'].apply(lambda x: returnFinalWord(x))

    return address[address_column]

In [36]:
delivery_df['buyeraddress'] = convert_address(delivery_df, 'buyeraddress')
delivery_df['selleraddress'] = convert_address(delivery_df, 'selleraddress')

In [37]:
delivery_df.head()

,orderid,pick,deliver_attempt_1,deliver_attempt_2,buyeraddress,selleraddress
0,2215676524,2020-03-02,2020-03-05,NaT,0,0
1,2219624609,2020-03-04,2020-03-06,2020-03-10,0,0
2,2220979489,2020-03-04,2020-03-06,NaT,0,0
3,2221066352,2020-03-05,2020-03-07,NaT,0,0
4,2222478803,2020-03-04,2020-03-06,NaT,1,0


# Processing Data

In [38]:
sla_matrix = [
             [3, 5, 7, 7],
             [5, 5, 7, 7],
             [7, 7, 7, 7],
             [7, 7, 7, 7]
]


In [41]:
def sla_max(buyeraddress, selleraddress):
    return sla_matrix[buyeraddress][selleraddress]

max_day = np.vectorize(sla_max)

In [43]:
delivery_df['sla'] = max_day(delivery_df['buyeraddress'], delivery_df['selleraddress'])
delivery_df

,orderid,pick,deliver_attempt_1,deliver_attempt_2,buyeraddress,selleraddress,sla
0,2215676524,2020-03-02,2020-03-05,NaT,0,0,3
1,2219624609,2020-03-04,2020-03-06,2020-03-10,0,0,3
2,2220979489,2020-03-04,2020-03-06,NaT,0,0,3
3,2221066352,2020-03-05,2020-03-07,NaT,0,0,3
4,2222478803,2020-03-04,2020-03-06,NaT,1,0,5
...,...,...,...,...,...,...,...
3176308,31504087640510,2020-04-02,2020-04-04,NaT,0,0,3
3176309,31504147352227,2020-04-03,2020-04-06,2020-04-07,0,0,3
3176310,31504462290482,2020-04-02,2020-04-04,NaT,0,0,3
3176311,31504851495943,2020-04-03,2020-04-07,NaT,1,0,5


In [45]:
holidays = ['2020-03-08', '2020-03-25', '2020-03-30', '2020-03-31']
weekmask = 'Mon Tue Wed Thu Fri Sat'

In [47]:
def dateSRS_to_arr(srs):
    return [np.datetime64('1970-01-01') if pd.isnull(x) else x.date() for x in srs]

In [49]:
pick_arr = dateSRS_to_arr(delivery_df['pick'])
one_delivery_attempt_arr = dateSRS_to_arr(delivery_df['deliver_attempt_1'])
two_delivery_attempt_arr = dateSRS_to_arr(delivery_df['deliver_attempt_2'])

delivery_df['attempt_1_duration'] = np.busday_count(pick_arr, one_delivery_attempt_arr, weekmask, holidays) 
delivery_df['attempt_2_duration'] = np.busday_count(one_delivery_attempt_arr, two_delivery_attempt_arr, weekmask, holidays)
delivery_df['is_late'] = (delivery_df['attempt_1_duration'] > delivery_df['sla']) | (delivery_df['attempt_2_duration'] > 3)
delivery_df['is_late'] = delivery_df['is_late'].astype(int) 

In [ ]:
#JUNK CODE -- TO BE USED IF YOU WANT TO MAKE YOUR LIFE COMPLICATED

"""def is_late_process(pick, one_deliver_attempt, two_deliver_attempt, buyeraddress, selleraddress):
    #pick = row.pick
    #one_deliver_attempt = row.deliver_attempt_1
    #two_deliver_attempt = row.deliver_attempt_2
    #buyeraddress = row.buyeraddress
    #selleraddress = row.selleraddress
    pick = pd.to_datetime(pick)
    one_deliver_attempt = pd.to_datetime(one_deliver_attempt)
    two_deliver_attempt = pd.to_datetime(two_deliver_attempt)

    #print(type(two_deliver_attempt))
    #print(f"buyeraddress[TYPE]: {type(buyeraddress)}")
    #print(f"selleraddress[TYPE]: {type(selleraddress)}")
    if type(two_deliver_attempt) != nat:
        #print("i went here!!!!!!")
        duration = (two_deliver_attempt - one_deliver_attempt) / np.timedelta64(1, 'D')
        #print(f"BEFORE duration process: \nduration: {duration}, duration[TYPE]: {type(duration)}")
        days_to_exclude = 1
        for i in exclude_date:
            if one_deliver_attempt < pd.to_datetime(i) < two_deliver_attempt:
                days_to_exclude += 1
        duration -= days_to_exclude
        #print(f"AFTER duration process: \nduration: {duration}, duration[TYPE]: {type(duration)}")
    else:
        #print(f"I went here instead lolloololol")
        duration = (one_deliver_attempt - pick) / np.timedelta64(1, 'D')
        #print(duration)
        days_to_exclude = 1 
        for i in exclude_date:
            if pick < pd.to_datetime(i) < one_deliver_attempt:
                days_to_exclude += 1
        duration -= days_to_exclude

    if duration < what_max_day(locations, buyeraddress, selleraddress):
        return 0
    else:
        return 1"""

In [ ]:
#result_df = pd.DataFrame()
#result_df['orderid'] = delivery_df['orderid']
#is_late = np.vectorize(is_late_process) -- Use if junk code is used

In [ ]:
#result_df['is_late'] = is_late(delivery_df['pick'].values, delivery_df['deliver_attempt_1'].values, delivery_df['deliver_attempt_2'], delivery_df['buyeraddress'], delivery_df['selleraddress'])
#result_df['is_late'] = delivery_df.apply(is_late, axis=1)

# Use this code if junk code is used. WARNING: WILL TAKE 4040 SECONDS TO PROCESS DATA.
# Accuracy: 63%

In [50]:
result_df = pd.DataFrame()
result_df['orderid'] = delivery_df['orderid']
result_df['is_late'] = delivery_df['is_late']

In [54]:
print(f"Are the delivery_df and result_df have the same number of rows? {delivery_df.shape[0] == result_df.shape[0]}")

Are the delivery_df and result_df have the same number of rows? True


In [53]:
result_df.to_csv('submission.csv', index=False)